# Представление о векторах

Если есть двумерное пространство, есть оси X и Y.

Прямую можно провести через 2 точки, поэтому нам нужны 2 точки, чтобы провести через них вектор.

Вектора для слов находятся в пространстве в большем количестве измерений (трех-, четырехмерное и тд)

Вектор – это ряд чисел, каждое число – это координата. Есть какое-то число, которое задает положение вектора в пространстве.

Вектор1     5  6  7  1
Вектор2     3  4  5  6 (Пример вектора в четырехмерном пространстве)

Математика может найти расстояние между векторами и узнать, насколько они близки.

Мы можем сравнивать только вектора, которые существует в одном пространстве (нельзя брать вектор из 
трехмерного и сравнивать с вектором из четырехмерного)

Векторизовать можно слова, тексты, корпуса (на самом деле что угодно). 

Для текста можно сделать так, что каждое слово станет измерением (пример)

            и  в  по над
Вектор1     5  6  7  1
Вектор2     3  4  5  6

Так, мы можем понять, насколько похожи эти вещи (устраиваем, например, из каждого слова их вектор
в зависимости от контекста)

Сколько раз 1 слово встречается с другими словами в тексте:

           Слово_n1 Слово_n2 Слово_n3  Слово_n4
Слово1     5        6        7         1
Слово2     3        4        5         6

В реальности в такой матрице будут почти все числа 0 и некоторое количество 1.
Есть алгоритмы, которые приводят такие матрицы к результату, как выше.

Алгоритм превращения слова в вектор – word2vec. Самый удобный пакет для этого – gensim

In [17]:
import gensim
import logging #он умеет сохранять и показывать разные штуки во время обучения модели

logging.basicConfig(format='%asctimes : %levelnames : %messages', logging=logging.INFO)
#рассказывает питону, что именно показывать на экране, когда модель обучается
#в интернете нет двоеточий в похожем коде

Каждое предложение во входном файле должно быть на отдельной строке.

Что можно сделать полезного: выкинуть служебные части речи, предварительно лемматизировать (если это мешает нашей задаче)

Кстати, gensim читает все сам сразу, потому что, если данные очень большие, то память компьютера забьется.

In [18]:
f = "war_and_peace.txt"
data = gensim.models.word2vec.LineSentence(f) #делает отдельное предложение на отдельной строке
model = gensim.models.Word2Vec(data, size=300, window=10, min_count=2, sg=0)

#size – размер вектора, даже размер 300 или 500 может быть хорошо информативным
#min_count – в нашу модель попадутся слова, которые имеют такую минимальную частотность
#sg=0 – skipgram

Мы обучили модель, дальше сохраним ее в виде файла

In [6]:
model.init_sims(replace=True)
model.save("my_model.model")

In [7]:
print(len(model.wv.vocab))
#узнать сколько слов

31345


In [15]:
word = "женщина"
if word in model:
    for element in model.wv.most_similar(positive=[word], topn=10):
        print(element)
#сначала показывается слово, а потом – косинусное значение

('него.', 0.9981270432472229)
('бежать', 0.9980776309967041)
('один', 0.9979787468910217)
('Наполеон', 0.9979562163352966)
('поразило', 0.997809886932373)
('ней,', 0.9977809190750122)
('покраснела', 0.9977301955223083)
('за', 0.997718095779419)
('ней', 0.9976932406425476)
('мужа,', 0.9976681470870972)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  


In [9]:
print(model.wv.doesnt_match("россия знает".split()))

#вот это работает норм
print(model.wv.doesnt_match("россия"))


2020-01-22 20:53:04,079 : WARNING : vectors for words {'россия'} are not present in the model, ignoring these words
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
2020-01-22 20:53:04,094 : WARNING : vectors for words {'р'} are not present in the model, ignoring these words


знает
я


In [16]:
word = "мужчины"
if word in model:
    for element in model.wv.most_similar(positive=["император", "мундир"], negative=["Наполеон"]):
        print(element)


('Война', 0.9989137649536133)
('является', 0.9988164901733398)
('zu', 0.998771607875824)
('Французский', 0.9987686276435852)
('левым', 0.9987384080886841)
('землю.', 0.9987369179725647)
('огромном', 0.9987170696258545)
('октября', 0.998698890209198)
('прекрасные', 0.998680591583252)
('клуба', 0.9986500144004822)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  


Берем дампы с Вики: dumps.wikimedia.org/backup-index.html

wiki-dump-parser #нужен pip install

Задание на сегодня:

1) Взять 2 корпуса текстов на одном языке (можно взять наши газеты)
2) Обработать вики парсером, разбить текст на предложения
3) На каждом построить модель
4) Сравнить составы квазисинонимов для нескольких слов (похожие или разные)

In [11]:
import wiki_dump_parser as parser
parser.xml_to_csv('dump.xml')

df = pd.read_csv('dump.csv', quotechar='|', index_col = False)
df['timestamp'] = pd.to_datetime(df['timestamp'],format='%Y-%m-%dT%H:%M:%SZ')

FileNotFoundError: [Errno 2] No such file or directory: 'dump.xml'